# Udacity - Machine Learning Engineer Nanodegree
## Capstone Project
### Title: Development of a LSTM Network to Predict Students’ Answers on Exam Questions

### Implementation of DKT:
#### Part 1: Define constants

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

fn = "data/ASSISTments_skill_builder_data.csv" # Dataset path
verbose = 1 # Verbose = {0,1,2}
best_model_weights = "weights/bestmodel" # File to save the model.
log_dir = "logs" # Path to save the logs.
optimizer = "adam" # Optimizer to use
lstm_units = 100 # Number of LSTM units
batch_size = 32 # Batch size
epochs = 10 # Number of epochs to train
dropout_rate = 0.3 # Dropout rate
test_fraction = 0.2 # Portion of data to be used for testing
validation_fraction = 0.2 # Portion of training data to be used for validation

#### Part 2: Pre-processing

In [2]:
from deepkt import deepkt, data_util, metrics


dataset, length, nb_features, nb_skills = data_util.load_dataset(fn=fn,
                                                                 batch_size=batch_size,
                                                                 shuffle=True)

train_set, test_set, val_set = data_util.split_dataset(dataset=dataset,
                                                       total_size=length,
                                                       test_fraction=test_fraction,
                                                       val_fraction=validation_fraction)


set_sz = length * batch_size
test_set_sz = (set_sz * test_fraction)
val_set_sz = (set_sz - test_set_sz) * validation_fraction
train_set_sz = set_sz - test_set_sz - val_set_sz
print("============= Data Summary =============")
print("Total number of students: %d" % set_sz)
print("Training set size: %d" % train_set_sz)
print("Validation set size: %d" % val_set_sz)
print("Testing set size: %d" % test_set_sz)
print("Number of skills: %d" % nb_skills)
print("Number of features in the input: %d" % nb_features)
print("========================================")

/Users/maywzh/.pyenv/versions/3.8.6/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
============= Data Summary =============
Total number of students: 4032
Training set size: 2580
Validation set size: 645
Testing set size: 806
Number of skills: 123
Number of features in the input: 246


In [3]:
train_set

<SkipDataset shapes: ((32, None, None), (32, None, None)), types: (tf.float32, tf.float32)>

In [4]:
 df = pd.read_csv(fn)

/Users/maywzh/.pyenv/versions/3.8.6/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,0,3,32454,30799,NaN,26,0.0,NaN,1.0,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,0,3,4922,30799,NaN,55,0.0,NaN,2.0,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,0,3,42000,30799,NaN,88,0.0,NaN,1.0,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,0,3,4859,30059,NaN,41,0.0,NaN,2.0,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,3,4,124564,30060,NaN,65,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525529,38214014,291495,96299,57830,108976,0,1,1,38234,tutor,...,0,2,38234,55692,200260.0,NaN,0.0,NaN,54.0,NaN
525530,38214016,291495,96299,57843,109015,0,1,1,6500,tutor,...,0,0,6500,55693,200299.0,NaN,0.0,NaN,55.0,NaN
525531,38214195,291503,96299,34577,54060,0,1,1,18344,tutor,...,0,3,18344,30677,NaN,0.8,0.0,NaN,56.0,NaN
525532,38214196,291503,96299,34577,54061,0,1,1,12922,tutor,...,0,2,12922,30677,NaN,-6,0.0,NaN,57.0,NaN


In [6]:
if "skill_id" not in df.columns:
    raise KeyError(f"The column 'skill_id' was not found on {fn}")
if "correct" not in df.columns:
    raise KeyError(f"The column 'correct' was not found on {fn}")
if "user_id" not in df.columns:
    raise KeyError(f"The column 'user_id' was not found on {fn}")

In [7]:
if not (df['correct'].isin([0, 1])).all():
        raise KeyError(f"The values of the column 'correct' must be 0 or 1.")

In [8]:
  df.dropna(subset=['skill_id'], inplace=True)
  df

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,0,3,32454,30799,NaN,26,0.0,NaN,1.0,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,0,3,4922,30799,NaN,55,0.0,NaN,2.0,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,0,3,42000,30799,NaN,88,0.0,NaN,1.0,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,0,3,4859,30059,NaN,41,0.0,NaN,2.0,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,3,4,124564,30060,NaN,65,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459203,33150408,277900,85730,69042,128442,1,0,2,51016,tutor,...,0,4,102125,59998,NaN,03/jul,0.0,NaN,13.0,13.0
459204,33150487,277900,85730,69056,128456,1,1,1,61547,tutor,...,0,4,61547,59998,NaN,2-May,0.0,NaN,14.0,14.0
459205,33150779,277900,85730,69046,128446,1,1,1,53890,tutor,...,0,4,53890,59998,NaN,10/-6,0.0,NaN,15.0,15.0
459206,33151098,277901,85730,69057,128457,1,0,4,181532,tutor,...,3,3,196844,60008,NaN,57/9,0.0,1.0,16.0,16.0


In [9]:
df = df.groupby('user_id').filter(lambda q: len(q) > 1).copy()

In [10]:
df

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,0,3,32454,30799,NaN,26,0.0,NaN,1.0,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,0,3,4922,30799,NaN,55,0.0,NaN,2.0,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,0,3,42000,30799,NaN,88,0.0,NaN,1.0,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,0,3,4859,30059,NaN,41,0.0,NaN,2.0,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,3,4,124564,30060,NaN,65,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459202,33150328,277900,85730,69050,128450,1,0,2,51594,tutor,...,0,4,130875,59998,NaN,0,0.0,NaN,12.0,12.0
459203,33150408,277900,85730,69042,128442,1,0,2,51016,tutor,...,0,4,102125,59998,NaN,03/jul,0.0,NaN,13.0,13.0
459204,33150487,277900,85730,69056,128456,1,1,1,61547,tutor,...,0,4,61547,59998,NaN,2-May,0.0,NaN,14.0,14.0
459205,33150779,277900,85730,69046,128446,1,1,1,53890,tutor,...,0,4,53890,59998,NaN,10/-6,0.0,NaN,15.0,15.0


In [11]:
df_exercise = df.groupby('exercise_id').filter(lambda q: len(q) == 2).copy()

KeyError: 'exercise_id'

In [51]:
df['skill'], _ = pd.factorize(df['skill_id'], sort=True)
#df['exercise'], _ = pd.factorize(df['problem_id'], sort=True)

In [49]:
df

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,0,3,32454,30799,NaN,26,0.0,NaN,1.0,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,0,3,4922,30799,NaN,55,0.0,NaN,2.0,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,0,3,42000,30799,NaN,88,0.0,NaN,1.0,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,0,3,4859,30059,NaN,41,0.0,NaN,2.0,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,3,4,124564,30060,NaN,65,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459202,33150328,277900,85730,69050,128450,1,0,2,51594,tutor,...,0,4,130875,59998,NaN,0,0.0,NaN,12.0,12.0
459203,33150408,277900,85730,69042,128442,1,0,2,51016,tutor,...,0,4,102125,59998,NaN,03/jul,0.0,NaN,13.0,13.0
459204,33150487,277900,85730,69056,128456,1,1,1,61547,tutor,...,0,4,61547,59998,NaN,2-May,0.0,NaN,14.0,14.0
459205,33150779,277900,85730,69046,128446,1,1,1,53890,tutor,...,0,4,53890,59998,NaN,10/-6,0.0,NaN,15.0,15.0


In [52]:
df['skill_with_answer'] = df['skill'] * 2 + df['correct']

In [76]:
df

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original,skill,skill_with_answer
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,32454,30799,NaN,26,0.0,NaN,1.0,1.0,0,1
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,4922,30799,NaN,55,0.0,NaN,2.0,2.0,0,1
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,42000,30799,NaN,88,0.0,NaN,1.0,1.0,0,0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,4859,30059,NaN,41,0.0,NaN,2.0,2.0,0,1
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,124564,30060,NaN,65,0.0,0.0,3.0,3.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459202,33150328,277900,85730,69050,128450,1,0,2,51594,tutor,...,130875,59998,NaN,0,0.0,NaN,12.0,12.0,122,244
459203,33150408,277900,85730,69042,128442,1,0,2,51016,tutor,...,102125,59998,NaN,03/jul,0.0,NaN,13.0,13.0,122,244
459204,33150487,277900,85730,69056,128456,1,1,1,61547,tutor,...,61547,59998,NaN,2-May,0.0,NaN,14.0,14.0,122,245
459205,33150779,277900,85730,69046,128446,1,1,1,53890,tutor,...,53890,59998,NaN,10/-6,0.0,NaN,15.0,15.0,122,245


In [13]:
dfg = df.groupby("problem_id")
dfg.size()

problem_id
83          4
84          4
85          5
86         30
249         1
         ... 
189408    210
189554    420
189565    210
189566    210
196456      1
Length: 17748, dtype: int64

In [25]:
gs = [g[0] for g in list(dfg)] 
gs
qk = {}
for g in gs:
    gg =dfg.get_group(g)
    qk[g] = gg["skill_id"].unique()

In [30]:
qk

47892: array([ 10., 104.]),
 47896: array([ 10., 104.]),
 47900: array([ 10., 104.]),
 48054: array([10., 64.]),
 48058: array([10., 64.]),
 48062: array([10., 64.]),
 48066: array([10., 64.]),
 48070: array([10., 64.]),
 48074: array([10., 64.]),
 48078: array([76.]),
 48082: array([10., 64.]),
 48086: array([10., 64.]),
 48090: array([10., 64.]),
 48094: array([10., 64.]),
 48098: array([10., 64.]),
 48102: array([10., 64.]),
 48106: array([10., 64.]),
 48110: array([10., 64.]),
 48114: array([10., 64.]),
 48118: array([10., 64.]),
 48122: array([10., 64.]),
 48126: array([10., 64.]),
 48130: array([10., 64.]),
 48134: array([10., 64.]),
 48138: array([10., 64.]),
 48142: array([10., 64.]),
 48146: array([10., 64.]),
 48150: array([10., 64.]),
 48154: array([10., 64.]),
 48158: array([10., 64.]),
 48162: array([10., 64.]),
 48166: array([10., 64.]),
 48170: array([10., 64.]),
 48446: array([74., 92.]),
 48447: array([74., 92.]),
 48448: array([74., 92.]),
 48449: array([74., 92.]),
 

In [44]:
df.loc[(df['problem_id'] == 48066)][["user_id","problem_id","skill_id","overlap_time"]]


,user_id,problem_id,skill_id,overlap_time
20436,77994,48066,10.0,43252
20698,78046,48066,10.0,34991
20760,78051,48066,10.0,13720
20849,78418,48066,10.0,29611
20921,78452,48066,10.0,31405
20964,78455,48066,10.0,18023
20972,78456,48066,10.0,58617
20983,78457,48066,10.0,34463
21987,78989,48066,10.0,98262
22606,79066,48066,10.0,17728


In [23]:
len(dfg.get_group(83)["skill_id"].unique())

1

In [12]:
g1 = dfg.get_group("83")
g1

NameError: name 'dfg' is not defined

AttributeError: 'DataFrame' object has no attribute 'unique'

{(83, 63.0): [161864, 161871, 161894, 162020], (84, 79.0): [220560, 220566, 221073, 222753], (85, 48.0): [127938, 127941, 127945, 127957, 129325], (86, 48.0): [127937, 127939, 127940, 127943, 127944, 127947, 127956, 127958, 127962, 129324], (86, 63.0): [161863, 161865, 161866, 161868, 161870, 161875, 161893, 161895, 161901, 162019], (86, 79.0): [220559, 220561, 220562, 220564, 220565, 220568, 221072, 221074, 221078, 222752], (249, 63.0): [161900], (250, 79.0): [221077], (251, 48.0): [127961], (252, 48.0): [127942, 127946, 127948, 127959, 127960], (252, 63.0): [161867, 161874, 161876, 161898, 161899], (252, 70.0): [173357, 173363, 185108, 186770, 186777], (252, 79.0): [220563, 220567, 221046, 221075, 221076], (807, 70.0): [173348, 173353, 173367, 186761, 186765, 186772, 186779, 190036], (808, 70.0): [173349, 173354, 186762, 186766, 186773, 186780, 190037], (809, 70.0): [173350, 173355, 173368, 186763, 186767, 186774, 186781, 190038], (810, 70.0): [173347, 173352, 173364, 173366, 186760,

In [29]:
seq = df.groupby('user_id').apply(
    lambda r: (
        r['skill_with_answer'].values[:-1],
        r['skill'].values[1:],
        r['exercise'].values[1:],
        r['problem_id'].values[1:],
        r['correct'].values[1:],
    )
)

In [32]:
seq[51950]

(array([196, 197, 197, 197, 199]),
 array([98, 98, 98, 99, 99]),
 array([13942, 13826, 13921, 14505, 14480]),
 array([ 97671,  97555,  97650, 108102, 108062]),
 array([1, 1, 1, 1, 1]))

In [30]:
seq[6452]

KeyError: 6452

In [16]:
    nb_users = len(seq)

In [17]:
nb_users

4047

In [18]:
dataset = tf.data.Dataset.from_generator(
        generator=lambda: seq,
        output_types=(tf.int32, tf.int32, tf.float32)
    )

In [19]:
dataset

<FlatMapDataset shapes: (<unknown>, <unknown>, <unknown>), types: (tf.int32, tf.int32, tf.float32)>

In [20]:
dataset

<FlatMapDataset shapes: (<unknown>, <unknown>, <unknown>), types: (tf.int32, tf.int32, tf.float32)>

In [21]:
dataset = dataset.shuffle(buffer_size=nb_users)

In [22]:
dataset = dataset.map(
    lambda feat, skill, label: (
        tf.one_hot(feat, depth=features_depth),
        tf.concat(
            values=[
                tf.one_hot(skill, depth=skill_depth),
                tf.expand_dims(label, -1)
            ],
            axis=-1
        )
    )
)

NameError: in user code:

    <ipython-input-22-2b3f66542271>:2 None  *
        )

    NameError: name 'features_depth' is not defined


#### Part 3: Building the model

In [3]:
student_model = deepkt.DKTModel(
                        nb_features=nb_features,
                        nb_skills=nb_skills,
                        hidden_units=lstm_units,
                        dropout_rate=dropout_rate)

student_model.compile(
        optimizer=optimizer,
        metrics=[
            metrics.BinaryAccuracy(),
            metrics.AUC(),
            metrics.Precision(),
            metrics.Recall()
        ])

student_model.summary()

Model: "DKTModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, None, 246)]       0         
_________________________________________________________________
masking (Masking)            (None, None, 246)         0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 100)         138800    
_________________________________________________________________
outputs (TimeDistributed)    (None, None, 123)         12423     
Total params: 151,223
Trainable params: 151,223
Non-trainable params: 0
_________________________________________________________________


#### Part 4: Train the Model

In [4]:
import tensorflow as tf
history = student_model.fit(dataset=train_set,
                            epochs=epochs,
                            verbose=verbose,
                            validation_data=val_set,
                            callbacks=[ 
                                tf.keras.callbacks.CSVLogger(f"{log_dir}/train.log"),
                                tf.keras.callbacks.ModelCheckpoint(best_model_weights,
                                                                   save_best_only=True,
                                                                   save_weights_only=True),
                                tf.keras.callbacks.TensorBoard(log_dir=log_dir)
        ])

Epoch 1/10
83/83 [==============================] - 12s 146ms/step - loss: 0.0657 - binary_accuracy: 0.7288 - auc: 0.7076 - precision: 0.7478 - recall: 0.9222 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
83/83 [==============================] - 9s 108ms/step - loss: 0.0557 - binary_accuracy: 0.7720 - auc: 0.8120 - precision: 0.7765 - recall: 0.9440 - val_loss: 0.0666 - val_binary_accuracy: 0.7878 - val_auc: 0.8344 - val_precision: 0.7908 - val_recall: 0.9487
Epoch 3/10
83/83 [==============================] - 9s 108ms/step - loss: 0.0612 - binary_accuracy: 0.7788 - auc: 0.8233 - precision: 0.7862 - recall: 0.9348 - val_loss: 0.0607 - val_binary_accuracy: 0.7715 - val_auc: 0.8289 - val_precision: 0.7733 - val_recall: 0.9310
Epoch 4/10
83/83 [==============================] - 9s 103ms/step - loss: 0.0596 - binary_accuracy: 0.7877 - auc: 0.8315 - precision: 0.7991 - recall: 0.9319 - val_loss:

#### Part 5: Load the Model with the Best Validation Loss

In [5]:
student_model.load_weights(best_model_weights)

#### Part 6: Test the Model

In [6]:
result = student_model.evaluate(test_set, verbose=verbose)

     26/Unknown - 2s 67ms/step - loss: 0.0529 - binary_accuracy: 0.8071 - auc: 0.8599 - precision: 0.8176 - recall: 0.9329